<a href="https://colab.research.google.com/github/Namesakenberg/deep_learning/blob/main/Feature__extraction_with_augmentation_pretrained_2_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import tensorflow
from tensorflow import keras
from tensorflow.keras import Sequential
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Dense , Flatten

# Load the data

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [4]:
!kaggle datasets download -d salader/dogs-vs-cats

Dataset URL: https://www.kaggle.com/datasets/salader/dogs-vs-cats
License(s): unknown
 97% 1.03G/1.06G [00:05<00:00, 157MB/s]
100% 1.06G/1.06G [00:05<00:00, 215MB/s]


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/dogs-vs-cats.zip' ,'r')
zip_ref.extractall('/content')
zip_ref.close()

# Get the pretrained model

In [6]:
conv_base = VGG16(
    include_top= False,
    weights='imagenet',
    input_shape=(150,150,3)
)

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


In [7]:
model = Sequential()
model.add(conv_base)
model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dense(1,activation='sigmoid'))
conv_base.trainable = False
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 4, 4, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 8192)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,048,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 15,763,521 (60.13 MB)

 Trainable params: 1,048,833 (4.00 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Use augmentation to make new data from existing ones to reduce overfitting

In [16]:
# data augumentation
train_datagen =ImageDataGenerator(
    rescale = 1./255,
    shear_range=0.2,
    zoom_range = 0.2,
    horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    directory = '/content/train',
    class_mode='binary',
    batch_size=32,
    target_size = (150 , 150)
)

validation_generator = test_datagen.flow_from_directory(
    directory = '/content/test',
    class_mode='binary',
    batch_size=32,
    target_size = (150 , 150)
)


Found 20000 images belonging to 2 classes.
Found 5000 images belonging to 2 classes.


In [17]:
model.compile(optimizer='adam',loss= 'binary_crossentropy' , metrics = ['accuracy'])

In [18]:
history = model.fit(
    train_generator,epochs = 10 , validation_data = validation_generator
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 172s 260ms/step - accuracy: 0.8274 - loss: 0.3768 - val_accuracy: 0.9056 - val_loss: 0.2208
Epoch 2/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 156s 250ms/step - accuracy: 0.8958 - loss: 0.2473 - val_accuracy: 0.9130 - val_loss: 0.2036
Epoch 3/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 252ms/step - accuracy: 0.9053 - loss: 0.2201 - val_accuracy: 0.9136 - val_loss: 0.2047
Epoch 4/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 160s 257ms/step - accuracy: 0.9079 - loss: 0.2114 - val_accuracy: 0.9144 - val_loss: 0.1991
Epoch 5/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 156s 249ms/step - accuracy: 0.9182 - loss: 0.1945 - val_accuracy: 0.9110 - val_loss: 0.2091
Epoch 6/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 157s 250ms/step - accuracy: 0.9162 - loss: 0.1969 - val_accuracy: 0.9082 - val_loss: 0.2203
Epoch 7/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 156s 249ms/step - accuracy: 0.9183 - loss: 0.1860 - val_accuracy: 0.9198 - val_loss: 0.1938
Epoch 8/10
625/625 ━━━━━━━━━━━━━━━━━━━━ 154s 247ms/step - accuracy: 0.9226 -